## Sentiment Analysis using LSTM

In [54]:
#Function to classify the statement as positive or negative
def predict_sentiment(text):
    tw = tokenizer.texts_to_sequences([text])
    tw = pad_sequences(tw,maxlen=200)
    prediction = int(model.predict(tw).round().item())
    print(sentiment_label[1][prediction], "statement")

In [48]:
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense, Dropout, SpatialDropout1D
from tensorflow.keras.layers import Embedding

#Read Data from csv file
df = pd.read_csv("./Data.csv")

In [49]:
#Process data for training
data_df = df[['text','sentiment']] #Extracting only text and sentiment from data
data_df = data_df[data_df['sentiment'] != 'neutral'] #Removing Neutral Sentiment
sentiment_label = data_df.sentiment.factorize() #Finding unique sentiment values
data = data_df.text.values
l = len(data)
data = data[0:l-1] #Removing NaN values in the data

#Tokenizing data into separate words
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(data)
vocab_size = len(tokenizer.word_index) + 1
encoded_docs = tokenizer.texts_to_sequences(data)
padded_sequence = pad_sequences(encoded_docs, maxlen=200)

In [50]:
#Describing model specifications for training
embedding_vector_length = 32
model = Sequential() 
model.add(Embedding(vocab_size, embedding_vector_length, input_length=200) )
model.add(SpatialDropout1D(0.25))
model.add(LSTM(50, dropout=0.5, recurrent_dropout=0.5))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid')) 
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])  

In [51]:
#Training the model
history = model.fit(padded_sequence,sentiment_label[0],validation_split=0.2, epochs=5, batch_size=32)

Epoch 1/5
289/289 [==============================] - 78s 259ms/step - loss: 0.3978 - accuracy: 0.8351 - val_loss: 0.2086 - val_accuracy: 0.9185
Epoch 2/5
289/289 [==============================] - 75s 258ms/step - loss: 0.2158 - accuracy: 0.9179 - val_loss: 0.1661 - val_accuracy: 0.9376
Epoch 3/5
289/289 [==============================] - 75s 261ms/step - loss: 0.1606 - accuracy: 0.9393 - val_loss: 0.1715 - val_accuracy: 0.9441
Epoch 4/5
289/289 [==============================] - 75s 260ms/step - loss: 0.1313 - accuracy: 0.9517 - val_loss: 0.1645 - val_accuracy: 0.9480
Epoch 5/5
289/289 [==============================] - 75s 259ms/step - loss: 0.1145 - accuracy: 0.9592 - val_loss: 0.1840 - val_accuracy: 0.9419


In [55]:
#Testing different cases
sentence1 = "It was fun"
predict_sentiment(sentence1)

sentence2 = "It was bad"
predict_sentiment(sentence2)

positive statement
negative statement
